In [34]:
import pandas as pd
import numpy as np

In [35]:
hotels = pd.read_csv('data/hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


**review_score to category**

In [36]:
counter = 0

average_score_preserved_mapper = dict()

for i in sorted(set(hotels['average_score'])):
    average_score_preserved_mapper.update({i:counter})
    counter += 1

print(average_score_preserved_mapper)

{5.2: 0, 6.4: 1, 6.6: 2, 6.7: 3, 6.8: 4, 6.9: 5, 7.0: 6, 7.1: 7, 7.2: 8, 7.3: 9, 7.4: 10, 7.5: 11, 7.6: 12, 7.7: 13, 7.8: 14, 7.9: 15, 8.0: 16, 8.1: 17, 8.2: 18, 8.3: 19, 8.4: 20, 8.5: 21, 8.6: 22, 8.7: 23, 8.8: 24, 8.9: 25, 9.0: 26, 9.1: 27, 9.2: 28, 9.3: 29, 9.4: 30, 9.5: 31, 9.6: 32, 9.8: 33}


In [37]:
hotels['average_score'] = hotels['average_score'].replace(average_score_preserved_mapper)
hotels['average_score'] = hotels['average_score'].astype(int)

**hotel_country**

In [38]:
countrified = pd.DataFrame()

def countrifier(string):
    cut = string.split()

    country = cut[-1:]

    country = country[0]

    if country == 'Kingdom':
        country = 'United Kingdom'
    return country

hotels['hotel_country'] = hotels['hotel_address'].apply(countrifier)

display(hotels['hotel_country'].value_counts())

United Kingdom    196774
Spain              45132
France             44830
Netherlands        43006
Austria            29178
Italy              27883
Name: hotel_country, dtype: int64

In [39]:
def ws_cutter(string):
    string = string[1:-1]
    return(string)

hotels['reviewer_nationality'] =\
    hotels['reviewer_nationality'].apply(ws_cutter)

In [40]:
hotels['is_homeland'] = (hotels['hotel_country']==hotels['reviewer_nationality'])
comp_pd = pd.DataFrame()
comp_pd['nat'], comp_pd['htl'], comp_pd['=='] =\
    hotels['reviewer_nationality'], hotels['hotel_country'], hotels['is_homeland']
display(hotels[['hotel_country', 'reviewer_nationality', 'is_homeland']].head(3))

,hotel_country,reviewer_nationality,is_homeland
0,United Kingdom,United Kingdom,True
1,United Kingdom,United Kingdom,True
2,France,China,False


**hotel_country categorize**

In [41]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 19 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  int32  
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

**days_since_review**

In [42]:
hotels['days_since_review'] = ((pd.to_datetime('2017-08-04')\
    -pd.to_datetime(hotels['review_date']))/ np.timedelta64(1, 'D'))
print(hotels['days_since_review'][0])

532.0


**effing tags (I really really hate 'em)**

In [43]:
def rev_func(num_tags):
    num_tags = num_tags[2:-2]
    res = num_tags.strip().split(' \', \' ')
    return res

hotels['tags_n'] = hotels['tags'].apply(rev_func)

**nights counter**

In [44]:
hotels['nights'] = 1
night_counter = pd.Series().astype(int)
counter = 0

for tags_listed in hotels['tags_n']:
    for i in tags_listed:
        if i[0:6] == 'Stayed':
            hotels['nights'][counter] = i[6:9]
    counter += 1

C:\Users\admin\AppData\Local\Temp\ipykernel_8348\4167505632.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  night_counter = pd.Series().astype(int)
C:\Users\admin\AppData\Local\Temp\ipykernel_8348\4167505632.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotels['nights'][counter] = i[6:9]


**cutting off insignificant amount of nights**

In [45]:
hotels['nights'] = hotels['nights'].astype(int)
hotels['nights'] = hotels['nights'].apply(lambda x: x if x < 12 else 13)
hotels['nights'].value_counts()

1     145519
2     100263
3      72000
4      35748
5      15611
6       7399
7       5549
8       1910
9        966
13       869
10       663
11       306
Name: nights, dtype: int64

In [46]:
hotels['tags'] = hotels['tags'].astype(str).str.lower()

def tag_detect(tags_listed, tag):
    if str(tag) in tags_listed:
            return True
    else: return False
        
tag_list = ['children', 'young', 'without', 'king', 'premier', 'standard', 'superior', 'luxury', 'leisure trip', 'solo', 'view']

for i in tag_list:
    hotels[i] = hotels['tags'].apply(tag_detect, tag = i)
    
# all_tags = list()

# for tag_list in hotels['tags_n']:
#     for i in tag_list:
#         i = i.split()
#         for k in i:
#             all_tags.append(k)

# all_tags = pd.Series(all_tags)

# pd.set_option("display.max_rows", None)
# display(all_tags.value_counts())

In [47]:
from collections import deque

get_city = pd.DataFrame()

def get_city(string):
    cut = deque(string.split()[-5:])
    city = cut[-2]
    if cut[-1] == 'Kingdom':
        city = cut[0]
    return pd.Series(city)

hotels['city'] = hotels['hotel_address'].apply(get_city)
display(hotels[['city']].describe())

,city
count,386803
unique,6
top,London
freq,196774


In [48]:
coord_dict = {'London': [51.507222, -0.1275],
 'Barcelona': [41.383333, 2.183333],
 'Paris': [48.856613, 2.352222],
 'Amsterdam': [52.366667, 4.9],
 'Vienna': [48.2, 16.366667],
 'Milan': [45.466944, 9.19]}

def coord_diff(city, lat, lng):
    if city in coord_dict:
        def_crd = coord_dict.get(city)

    lat_diff = def_crd[0] - lat
    lng_diff = def_crd[1] - lng
    return pd.Series([lat_diff, lng_diff])


hotels[['lat_diff','lng_diff']] = hotels.apply(lambda x: coord_diff(x['city'], x['lat'], x['lng']), axis=1)

In [49]:
# def citifier(string):
#     cut = string.split()[::-1]

#     country = cut[1]

#     country = country[0]

#     if country == 'Kingdom':
#         city = cut[3:]
#     else: city = cut[2:]
    
#     city = city[0]
    
#     return city

# hotels['hotel_city'] = hotels['hotel_address'].apply(citifier)

# display(hotels['hotel_city'].value_counts())

In [55]:
hotels['positive_review'] = hotels['positive_review'].astype(str).str.lower().str.strip()
hotels['negative_review'] = hotels['negative_review'].astype(str).str.lower().str.strip()

In [56]:
counts = hotels.positive_review.value_counts()
counts.to_csv('positive_review.csv')

counts = hotels.negative_review.value_counts()
counts.to_csv('negative_review.csv')

In [57]:
def tag_detect(tags_listed, tag):
    if str(tag) in tags_listed:
            return True
    else: return False
        
tag_list = ['location', 'everything', 'nothing', 'great location', 'the location', 'good location']

for i in tag_list:
    hotels['positive_'+i] = hotels['positive_review'].apply(tag_detect, tag = i)

In [58]:
def tag_detect(tags_listed, tag):
    if str(tag) in tags_listed:
            return True
    else: return False
        
tag_list = ['no positive', 'nothing']

for i in tag_list:
    hotels['no_positive'] = hotels['positive_review'].apply(tag_detect, tag = i)

Good location

In [181]:
def rev_func(num_tags):
    res = num_tags.strip().split(' \', \' ')
    return res

hotels['positive_review_n'] = hotels['positive_review'].apply(rev_func)

In [182]:
def tag_detect(tags_listed, tag):
    count = False
    if 'location' == tag:
            count = True
    return count

for i in tag_list:
    hotels['good_location'] = hotels['positive_review_n'].apply(tag_detect, tag = i)

In [193]:
for i in range(len(hotels.positive_review_n)):
    if 'location' in  hotels.positive_review_n[i]:
        hotels.loc[:, ('good_location', i)] = True

KeyboardInterrupt: 

In [195]:
hotels['good_location'].value_counts()

True    386803
Name: good_location, dtype: int64

In [199]:
hotels['good_location'][0]

True

In [198]:
hotels['positive_review_n'][0]

['staff were amazing']

In [200]:
type(hotels['positive_review_n'][1])

list

In [ ]:
def tag_detect(tags_listed, tag):
    if str(tag) in tags_listed:
            return True
    else: return False
        
tag_list = ['location', 'breakfast']

for i in tag_list:
    hotels['negative_'+i] = hotels['negative_review'].apply(tag_detect, tag = i)

In [ ]:
def tag_detect(tags_listed, tag):
    if str(tag) in tags_listed:
            return True
    else: return False
        
tag_list = ['location', 'no negative', 'n a', 'none',  'nothing', '', 'nothing at all', 'nothing to dislike', 'no complaints', 'all good', 'nil', 'na']

for i in tag_list:
    hotels['no_negative'] = hotels['negative_review'].apply(tag_detect, tag = i)

Bad location

In [ ]:
def tag_detect(tags_listed, tag):
    if 'location' in str(tag):
            return True
    else: return False

for i in tag_list:
    hotels['good_location'] = hotels['positive_review'].apply(tag_detect, tag = i)

Correlation test

In [52]:
hotels.corrwith(hotels['reviewer_score'])

additional_number_of_scoring                 -0.061773
average_score                                 0.363863
review_total_negative_word_counts            -0.383407
total_number_of_reviews                      -0.073389
review_total_positive_word_counts             0.220294
total_number_of_reviews_reviewer_has_given    0.002498
reviewer_score                                1.000000
days_since_review                             0.002408
lat                                          -0.036984
lng                                           0.034829
is_homeland                                   0.009279
nights                                       -0.014889
children                                     -0.014233
young                                        -0.020672
without                                      -0.056660
king                                          0.036716
premier                                       0.020603
standard                                     -0.060620
superior  

## ***IT'S A FINAL COUNTDOWN***

In [24]:
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
hotels.drop(object_columns, axis = 1, inplace=True)


hotels = hotels.fillna(0)

hotels.drop(['lat', 'lng', 'days_since_review','lat_diff',], axis=1, inplace=True)

In [25]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [26]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [27]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [28]:
X_test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 96701 entries, 177225 to 152014
Data columns (total 33 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   additional_number_of_scoring                96701 non-null  int64  
 1   average_score                               96701 non-null  int32  
 2   review_total_negative_word_counts           96701 non-null  int64  
 3   total_number_of_reviews                     96701 non-null  int64  
 4   review_total_positive_word_counts           96701 non-null  int64  
 5   total_number_of_reviews_reviewer_has_given  96701 non-null  int64  
 6   is_homeland                                 96701 non-null  bool   
 7   nights                                      96701 non-null  int64  
 8   children                                    96701 non-null  bool   
 9   young                                       96701 non-null  bool   
 10  with

In [29]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


MAPE: 0.14134178189311136

MAPE: 0.14131385806717026

MAPE: 0.14053156733726535

MAPE: 0.14070534667401732

MAPE: 0.1405255170406278

MAPE: 0.13755135308025937

MAPE: 0.13739329046723142

MAPE: 0.13732568793244715

MAPE: 0.13657435490890754

MAPE: 0.137793269782532

MAPE: 0.13801613260621556 minus homeland

MAPE: 0.13754144731719012 plus review tags

MAPE: 0.13699742123463077 divided review tags into pos\neg

MAPE: 0.13150680506034415 +review tags

In [30]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.13150680506034415


Небольшой бонус:


In [31]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)